In [1]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [2]:
# 경로 설정
data_path = '/Users/kyoungseo/lgupls_aistage/data/'
saved_path = '/Users/kyoungseo/lgupls_aistage/saved'
output_path = '/Users/kyoungseo/lgupls_aistage/submission'

In [3]:
# 데이터 불러오기 
history = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
watch = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
buy = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8') 
search = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
profile = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
meta_plus = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

In [5]:
hd = history.copy()
wd = watch.copy()
bd = buy.copy()
sd = search.copy()
prd = profile.copy()
md = meta.copy()
mpd = meta_plus.copy()

In [5]:
print('hd 데이터(중복 제거 전) : ', hd.shape)
print('wd 데이터(중복 제거 전) : ', wd.shape)

hd 데이터(중복 제거 전) :  (1005651, 8)
wd 데이터(중복 제거 전) :  (892794, 8)


#### 1. 중복 데이터 제거

In [6]:
# 중복행 확인
hd[hd.duplicated()] # 시청시작 데이터

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer
517,5,20220421082427,20220421082957,MKID003,92,NaN,N,N
519,5,20220421082427,20220421085009,MKID003,174,NaN,N,N
521,5,20220421082427,20220421090237,MKID003,201,NaN,N,N
523,5,20220421082427,20220421090332,MKID003,187,NaN,N,N
525,5,20220421082427,20220421090449,MKID003,305,NaN,N,N
...,...,...,...,...,...,...,...,...
1004498,32908,20220420203843,20220420204139,MKID003,6458,NaN,N,N
1005467,33015,20220424194027,20220424194036,MKID003,416,NaN,Y,N
1005469,33015,20220424194027,20220424194091,MKID003,175,NaN,N,Y
1005471,33015,20220424195608,20220424195612,MKID003,57,NaN,N,N


In [7]:
wd[wd.duplicated()] # 시청종료 데이터

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play
507,5,20220421082427,20220421082953,MKID049,298,307,309,1
509,5,20220421082427,20220421085005,MKID049,92,1207,1207,1
511,5,20220421082427,20220421090233,MKID049,174,744,745,1
513,5,20220421082427,20220421090327,MKID049,201,50,50,1
515,5,20220421082427,20220421090443,MKID049,187,70,70,1
...,...,...,...,...,...,...,...,...
892571,33016,20220424194881,20220424195327,MKID049,5873,94,94,1
892573,33016,20220424194881,20220424195543,MKID049,5874,94,94,1
892575,33016,20220424194881,20220424195570,MKID049,5874,50,94,0
892577,33016,20220424194881,20220424195683,MKID049,4598,34,95,0


In [8]:
# 중복행 제거
hd = hd[~hd.duplicated()]
print('hd 데이터(중복 제거 후) : ', hd.shape)

wd = wd[~wd.duplicated()]
print('wd 데이터(중복 제거 후) : ', wd.shape)

hd 데이터(중복 제거 후) :  (899273, 8)
wd 데이터(중복 제거 후) :  (800740, 8)


#### 2. 이상치 제거

#### 2-1. History_data 내 log_time 이상치 제거

In [14]:
hd.head()

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer,date,time
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N,20220301,115719
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N,20220301,115809
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N,20220301,115958
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N,20220301,120118
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N,20220301,120229


In [10]:
hd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899273 entries, 0 to 1005650
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       899273 non-null  int64  
 1   ss_id            899273 non-null  int64  
 2   log_time         899273 non-null  int64  
 3   act_target_dtl   899273 non-null  object 
 4   album_id         899273 non-null  int64  
 5   payment          59193 non-null   float64
 6   continuous_play  899273 non-null  object 
 7   short_trailer    899273 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 61.7+ MB


In [11]:
# log_time 날짜/시간 분리
hd["log_time"] = hd["log_time"].astype(str)
hd["date"] = hd["log_time"].str.slice(0, 8)
hd["time"] = hd["log_time"].str.slice(8, 14)

In [12]:
hd.head()

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer,date,time
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N,20220301,115719
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N,20220301,115809
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N,20220301,115958
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N,20220301,120118
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N,20220301,120229


In [13]:
hd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899273 entries, 0 to 1005650
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       899273 non-null  int64  
 1   ss_id            899273 non-null  int64  
 2   log_time         899273 non-null  object 
 3   act_target_dtl   899273 non-null  object 
 4   album_id         899273 non-null  int64  
 5   payment          59193 non-null   float64
 6   continuous_play  899273 non-null  object 
 7   short_trailer    899273 non-null  object 
 8   date             899273 non-null  object 
 9   time             899273 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 75.5+ MB


In [14]:
# 월 조건 (3 ~ 7월 제외한 월이 있는지)
con1 = (hd['date'].str[4:6] != '03')
con2 = (hd['date'].str[4:6] != '04')
con3 = (hd['date'].str[4:6] != '05')
con4 = (hd['date'].str[4:6] != '06')
con5 = (hd['date'].str[4:6] != '07')

# 일 조건 (0일 / 00일 / 32일 이상 있는지)
con8 = (hd['date'].str[6:8] == '0')
con9 = (hd['date'].str[6:8] == '00')
con10 = (hd['date'].str[6:8].astype(int) >= 32)

In [15]:
print('조건에 맞지 않는 연도 수:', len(hd.loc[hd['date'].str[:4] != '2022']))
print('조건에 맞지 않는 월 수:', len(hd.loc[con1 & con2 & con3 & con4 & con5]))
print('조건에 맞지 않는 일 수:', len(hd.loc[con8 | con9 | con10]))

조건에 맞지 않는 연도 수: 0
조건에 맞지 않는 월 수: 0
조건에 맞지 않는 일 수: 0


In [16]:
print('조건에 맞지 않는 시 수:', len(hd.loc[hd['time'].str[:2] >= '24']))
print('조건에 맞지 않는 분 수:', len(hd.loc[hd['time'].str[2:4] >= '60']))
print('조건에 맞지 않는 초 수:', len(hd.loc[hd['time'].str[4:6] >= '60']))

조건에 맞지 않는 시 수: 0
조건에 맞지 않는 분 수: 0
조건에 맞지 않는 초 수: 73581


In [17]:
# 초에만 이상 있음
# 이상 있는 데이터 73,581개
hd.loc[hd['time'].str[4:6] >= '60']

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer,date,time
2975,30,20220305130048,20220305131273,MKID003,1901,NaN,Y,N,20220305,131273
2976,30,20220305130048,20220305132966,MKID003,1902,NaN,Y,N,20220305,132966
2978,30,20220305130048,20220305134092,MKID003,1903,NaN,Y,N,20220305,134092
2983,30,20220320140092,20220320140174,MKID003,446,500.0,N,N,20220320,140174
2988,30,20220320140092,20220320144788,MKID003,1908,500.0,Y,N,20220320,144788
...,...,...,...,...,...,...,...,...,...,...
1005638,33032,20220427151499,20220427154870,MKID003,7105,NaN,N,N,20220427,154870
1005639,33032,20220427151499,20220427155075,MKID003,1725,NaN,Y,N,20220427,155075
1005646,33032,20220427155091,20220427155668,MKID003,381,NaN,Y,N,20220427,155668
1005647,33032,20220427155091,20220427155680,MKID003,381,NaN,Y,N,20220427,155680


In [18]:
# 이상 데이터 제거
hd = hd.drop(hd.loc[hd['time'].str[4:6] >= '60'].index)
print('이상 데이터 제거 후:', len(hd))

이상 데이터 제거 후: 825692


In [19]:
print('조건에 맞지 않는 초 수:', len(hd.loc[hd['time'].str[4:6] >= '60']))

조건에 맞지 않는 초 수: 0


#### 2-2. log_time 파생변수 생성

- 6시간 기준으로 나누기

- 참고문헌<br>
- 2개 이상의 조건이 있을 때<br>
https://www.delftstack.com/ko/howto/python-pandas/how-to-create-dataframe-column-based-on-given-condition-in-pandas/

- 새로운 변수 생성<br>
1) 00_06: log_time이 00-06시 사이<br>
2) 06_12: log_time이 06-12시 사이<br>
3) 12_18: log_time이 12-18시 사이<br>
4) 18_24: log_time이 18-24시 사이

In [20]:
# log_time에서 시간만 분리
hd["log_time"] = hd["log_time"].astype(str)
hd["time"] = hd["log_time"].str[8:10].astype(int)

In [21]:
conditionlist = [
    (hd['time'] < 6) ,
    (hd['time'] >= 6) & (hd['time'] < 12),
    (hd['time'] >= 12) & (hd['time'] < 18),
    (hd['time'] >= 18) & (hd['time'] < 24)]
choicelist = [1, 2, 3, 4]
hd['time_slot'] = np.select(conditionlist, choicelist)

In [22]:
hd.head(10)

,profile_id,ss_id,log_time,act_target_dtl,album_id,payment,continuous_play,short_trailer,date,time,time_slot
0,3,20220301115653,20220301115719,MKID003,15,NaN,Y,N,20220301,11,2
1,3,20220301115653,20220301115809,MKID003,16,NaN,Y,N,20220301,11,2
2,3,20220301115653,20220301115958,MKID003,17,NaN,Y,N,20220301,11,2
3,3,20220301115653,20220301120118,MKID003,18,NaN,Y,N,20220301,12,3
4,3,20220301115653,20220301120229,MKID003,19,NaN,Y,N,20220301,12,3
5,3,20220301115653,20220301120450,MKID003,20,NaN,Y,N,20220301,12,3
6,3,20220301115653,20220301120654,MKID003,21,NaN,Y,N,20220301,12,3
7,3,20220301115653,20220301120859,MKID003,22,NaN,Y,N,20220301,12,3
8,3,20220301115653,20220301121104,MKID003,23,NaN,Y,N,20220301,12,3
9,3,20220301115653,20220301121308,MKID003,24,NaN,Y,N,20220301,12,3


In [23]:
hd["date"] = hd["date"].astype(int)
hd["time"] = hd["log_time"].str.slice(8, 14)
hd["time"] = hd["time"].astype(int)

In [24]:
hd = hd.drop(['log_time'], axis = 1)

In [25]:
hd = hd[['profile_id', 'ss_id', 'date', 'time', 'time_slot', 'act_target_dtl', 'album_id', 'payment',
       'continuous_play', 'short_trailer']]
hd.head()

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301,115719,2,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301,115809,2,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301,115958,2,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301,120118,3,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301,120229,3,MKID003,19,NaN,Y,N


#### 2-3. watch_data 내 log_time 이상치 제거

In [26]:
# log_time 날짜/시간 분리
wd["log_time"] = wd["log_time"].astype(str)
wd["date"] = wd["log_time"].str.slice(0, 8)
wd["time"] = wd["log_time"].str.slice(8, 14)

In [27]:
wd.head()

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play,date,time
0,3,20220301115653,20220301115805,MKID049,15,46,46,1,20220301,115805
1,3,20220301115653,20220301115954,MKID049,16,104,105,1,20220301,115954
2,3,20220301115653,20220301120114,MKID049,17,76,76,1,20220301,120114
3,3,20220301115653,20220301120226,MKID049,18,67,68,1,20220301,120226
4,3,20220301115653,20220301120400,MKID049,19,90,90,0,20220301,120400


In [28]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800740 entries, 0 to 892793
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   profile_id       800740 non-null  int64 
 1   ss_id            800740 non-null  int64 
 2   log_time         800740 non-null  object
 3   act_target_dtl   800740 non-null  object
 4   album_id         800740 non-null  int64 
 5   watch_time       800740 non-null  int64 
 6   total_time       800740 non-null  int64 
 7   continuous_play  800740 non-null  int64 
 8   date             800740 non-null  object
 9   time             800740 non-null  object
dtypes: int64(6), object(4)
memory usage: 67.2+ MB


In [29]:
# 월 조건 (3 ~ 7월 제외한 월이 있는지)
con1 = (wd['date'].str[4:6] != '03')
con2 = (wd['date'].str[4:6] != '04')
con3 = (wd['date'].str[4:6] != '05')
con4 = (wd['date'].str[4:6] != '06')
con5 = (wd['date'].str[4:6] != '07')

# 일 조건 (0일 / 00일 / 32일 이상 있는지)
con8 = (wd['date'].str[6:8] == '0')
con9 = (wd['date'].str[6:8] == '00')
con10 = (wd['date'].str[6:8].astype(int) >= 32)

In [30]:
print('조건에 맞지 않는 연도 수:', len(wd.loc[wd['date'].str[:4] != '2022']))
print('조건에 맞지 않는 월 수:', len(wd.loc[con1 & con2 & con3 & con4 & con5]))
print('조건에 맞지 않는 일 수:', len(wd.loc[con8 | con9 | con10]))

조건에 맞지 않는 연도 수: 0
조건에 맞지 않는 월 수: 0
조건에 맞지 않는 일 수: 0


In [31]:
print('조건에 맞지 않는 시 수:', len(wd.loc[wd['time'].str[:2] >= '24']))
print('조건에 맞지 않는 분 수:', len(wd.loc[wd['time'].str[2:4] >= '60']))
print('조건에 맞지 않는 초 수:', len(wd.loc[wd['time'].str[4:6] >= '60']))

조건에 맞지 않는 시 수: 0
조건에 맞지 않는 분 수: 0
조건에 맞지 않는 초 수: 61392


In [32]:
# 초에만 이상 있음
# 이상 있는 데이터 61,392개
wd.loc[wd['time'].str[4:6] >= '60']

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play,date,time
2849,30,20220305130048,20220305131180,MKID049,1900,659,661,1,20220305,131180
2855,30,20220320140092,20220320140184,MKID049,1904,664,666,1,20220320,140184
2856,30,20220320140092,20220320141393,MKID049,1905,664,666,1,20220320,141393
2859,30,20220320140092,20220320144772,MKID049,1908,664,666,1,20220320,144772
2865,30,20220408224805,20220408225284,MKID049,1299,2,185,0,20220408,225284
...,...,...,...,...,...,...,...,...,...,...
892777,33032,20220427151499,20220427154969,MKID049,0,3,136,0,20220427,154969
892778,33032,20220427151499,20220427155080,MKID049,1725,4,106,0,20220427,155080
892781,33032,20220427155091,20220427155385,MKID049,393,81,1036,0,20220427,155385
892790,33032,20220427155091,20220427155694,MKID049,381,462,463,1,20220427,155694


In [33]:
# 이상 데이터 제거
wd = wd.drop(wd.loc[wd['time'].str[4:6] >= '60'].index)
print('이상 데이터 제거 후:', len(wd))

이상 데이터 제거 후: 739348


In [34]:
print('조건에 맞지 않는 초 수:', len(wd.loc[wd['time'].str[4:6] >= '60']))

조건에 맞지 않는 초 수: 0


#### 2-4. log_time 파생변수 생성

In [35]:
# log_time에서 시간만 분리
wd["log_time"] = wd["log_time"].astype(str)
wd["time"] = wd["log_time"].str[8:10].astype(int)

In [36]:
conditionlist = [
    (wd['time'] < 6) ,
    (wd['time'] >= 6) & (wd['time'] < 12),
    (wd['time'] >= 12) & (wd['time'] < 18),
    (wd['time'] >= 18) & (wd['time'] < 24)]
choicelist = [1, 2, 3, 4]
wd['time_slot'] = np.select(conditionlist, choicelist)

In [37]:
wd.head(10)

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play,date,time,time_slot
0,3,20220301115653,20220301115805,MKID049,15,46,46,1,20220301,11,2
1,3,20220301115653,20220301115954,MKID049,16,104,105,1,20220301,11,2
2,3,20220301115653,20220301120114,MKID049,17,76,76,1,20220301,12,3
3,3,20220301115653,20220301120226,MKID049,18,67,68,1,20220301,12,3
4,3,20220301115653,20220301120400,MKID049,19,90,90,0,20220301,12,3
5,3,20220301115653,20220301120650,MKID049,20,119,119,1,20220301,12,3
6,3,20220301115653,20220301120854,MKID049,21,119,119,1,20220301,12,3
7,3,20220301115653,20220301121059,MKID049,22,119,119,1,20220301,12,3
8,3,20220301115653,20220301121304,MKID049,23,120,121,1,20220301,12,3
9,3,20220301115653,20220301121508,MKID049,24,119,119,1,20220301,12,3


In [38]:
wd.tail(10)

,profile_id,ss_id,log_time,act_target_dtl,album_id,watch_time,total_time,continuous_play,date,time,time_slot
892782,33032,20220427155091,20220427155457,MKID049,3625,9,66,0,20220427,15,3
892783,33032,20220427155091,20220427155526,MKID049,375,658,659,1,20220427,15,3
892784,33032,20220427155091,20220427155529,MKID049,381,462,463,1,20220427,15,3
892785,33032,20220427155091,20220427155601,MKID049,381,462,463,1,20220427,15,3
892786,33032,20220427155091,20220427155603,MKID049,125,6,520,0,20220427,15,3
892787,33032,20220427155091,20220427155646,MKID049,381,462,463,1,20220427,15,3
892788,33032,20220427155091,20220427155648,MKID049,381,463,464,1,20220427,15,3
892789,33032,20220427155091,20220427155653,MKID049,381,463,464,1,20220427,15,3
892791,33032,20220427155839,20220427155826,MKID049,125,10,520,0,20220427,15,3
892792,33032,20220427155706,20220427155836,MKID049,125,6,521,0,20220427,15,3


In [39]:
wd["date"] = wd["date"].astype(int)
wd["time"] = wd["log_time"].str.slice(8, 14)
wd["time"] = wd["time"].astype(int)

In [40]:
wd = wd.drop(['log_time'], axis = 1)

In [41]:
wd = wd[['profile_id', 'ss_id', 'date', 'time', 'time_slot', 'act_target_dtl', 'album_id', 'watch_time',
       'total_time', 'continuous_play']]
wd.head()

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,watch_time,total_time,continuous_play
0,3,20220301115653,20220301,115805,2,MKID049,15,46,46,1
1,3,20220301115653,20220301,115954,2,MKID049,16,104,105,1
2,3,20220301115653,20220301,120114,3,MKID049,17,76,76,1
3,3,20220301115653,20220301,120226,3,MKID049,18,67,68,1
4,3,20220301115653,20220301,120400,3,MKID049,19,90,90,0


#### 3. profile data 결측값 처리

In [42]:
prd.head()

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,F,5,P02,P04,P07,K01,K03,K04
1,5,M,5,P07,P08,P06,K05,K08,K09
2,7,F,9,P05,P03,NaN,K06,K04,NaN
3,12,M,6,P03,P06,P02,K09,K07,K03
4,16,F,12,P03,P06,P01,K01,K06,K04


In [43]:
prd.shape

(8311, 9)

In [44]:
prd.isnull().sum()

profile_id                     0
sex                            0
age                            0
pr_interest_keyword_cd_1       0
pr_interest_keyword_cd_2    1533
pr_interest_keyword_cd_3    2080
ch_interest_keyword_cd_1       0
ch_interest_keyword_cd_2    1693
ch_interest_keyword_cd_3    2282
dtype: int64

* 결측값 있는 column<br>

1) pr_interest_keyword_cd_2(부모 관심 키워드2): 1533<br>
2) pr_interest_keyword_cd_3(부모 관심 키워드3): 2080<br>
3) ch_interest_keyword_cd_2(아이 관심 키워드2): 1693<br>
4) ch_interest_keyword_cd_3(아이 관심 키워드3): 2282<br>

- 키워드1과 동일한 값 2,3에 채우기

In [45]:
prd['pr_interest_keyword_cd_2'] = prd['pr_interest_keyword_cd_2'].fillna(prd['pr_interest_keyword_cd_1'])
prd['pr_interest_keyword_cd_3'] = prd['pr_interest_keyword_cd_3'].fillna(prd['pr_interest_keyword_cd_1'])
prd['ch_interest_keyword_cd_2'] = prd['ch_interest_keyword_cd_2'].fillna(prd['ch_interest_keyword_cd_1'])
prd['ch_interest_keyword_cd_3'] = prd['ch_interest_keyword_cd_3'].fillna(prd['ch_interest_keyword_cd_1'])

In [46]:
prd.head(10)

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,F,5,P02,P04,P07,K01,K03,K04
1,5,M,5,P07,P08,P06,K05,K08,K09
2,7,F,9,P05,P03,P05,K06,K04,K06
3,12,M,6,P03,P06,P02,K09,K07,K03
4,16,F,12,P03,P06,P01,K01,K06,K04
5,19,M,3,P05,P06,P04,K01,K08,K09
6,20,M,7,P07,P08,P03,K07,K01,K08
7,22,F,4,P06,P06,P06,K01,K01,K01
8,24,M,5,P05,P04,P07,K09,K01,K02
9,26,M,7,P01,P07,P03,K02,K06,K08


In [47]:
prd.isnull().sum()

profile_id                  0
sex                         0
age                         0
pr_interest_keyword_cd_1    0
pr_interest_keyword_cd_2    0
pr_interest_keyword_cd_3    0
ch_interest_keyword_cd_1    0
ch_interest_keyword_cd_2    0
ch_interest_keyword_cd_3    0
dtype: int64

#### 4. 범주형 데이터: 인코딩 정리

https://suy379.tistory.com/157<br>
https://jimmy-ai.tistory.com/263

#### 4-1. history data

In [48]:
hd.head()

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301,115719,2,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301,115809,2,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301,115958,2,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301,120118,3,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301,120229,3,MKID003,19,NaN,Y,N


- continuous_play, short_trailer: 이진변수
- 0, 1로 변경


* continuous_play
    - 연속재생x: 0
    - 연속재생o: 1
    
    
* short_trailer
    - N: 0
    - Y: 1

In [49]:
hd.loc[hd['short_trailer'] == 'Y']

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,payment,continuous_play,short_trailer
20,3,20220301115653,20220301,123505,3,MKID003,33,NaN,Y,Y
123,5,20220309185728,20220309,195543,4,MKID003,138,NaN,Y,Y
144,5,20220313130615,20220313,131521,3,MKID003,150,NaN,Y,Y
156,5,20220318212718,20220318,212809,4,MKID003,159,NaN,Y,Y
157,5,20220318212718,20220318,212811,4,MKID003,160,NaN,Y,Y
...,...,...,...,...,...,...,...,...,...,...
1005580,33032,20220427110951,20220427,111026,2,MKID003,4944,NaN,Y,Y
1005581,33032,20220427110951,20220427,111026,2,MKID003,3692,NaN,N,Y
1005592,33032,20220427112116,20220427,112305,2,MKID003,14665,NaN,Y,Y
1005593,33032,20220427112116,20220427,112335,2,MKID003,14662,NaN,Y,Y


In [50]:
hd.loc[hd['short_trailer'] == 'N']

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301,115719,2,MKID003,15,NaN,Y,N
1,3,20220301115653,20220301,115809,2,MKID003,16,NaN,Y,N
2,3,20220301115653,20220301,115958,2,MKID003,17,NaN,Y,N
3,3,20220301115653,20220301,120118,3,MKID003,18,NaN,Y,N
4,3,20220301115653,20220301,120229,3,MKID003,19,NaN,Y,N
...,...,...,...,...,...,...,...,...,...,...
1005643,33032,20220427155091,20220427,155601,3,MKID003,381,NaN,Y,N
1005644,33032,20220427155091,20220427,155611,3,MKID003,381,NaN,Y,N
1005645,33032,20220427155091,20220427,155642,3,MKID003,381,NaN,N,N
1005648,33032,20220427155839,20220427,155810,3,MKID003,125,NaN,Y,N


In [51]:
hd['short_trailer'] = hd['short_trailer'].apply(lambda x: 1 if x == 'Y' else 0)

In [52]:
hd.loc[hd['short_trailer'] == 1]

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,payment,continuous_play,short_trailer
20,3,20220301115653,20220301,123505,3,MKID003,33,NaN,Y,1
123,5,20220309185728,20220309,195543,4,MKID003,138,NaN,Y,1
144,5,20220313130615,20220313,131521,3,MKID003,150,NaN,Y,1
156,5,20220318212718,20220318,212809,4,MKID003,159,NaN,Y,1
157,5,20220318212718,20220318,212811,4,MKID003,160,NaN,Y,1
...,...,...,...,...,...,...,...,...,...,...
1005580,33032,20220427110951,20220427,111026,2,MKID003,4944,NaN,Y,1
1005581,33032,20220427110951,20220427,111026,2,MKID003,3692,NaN,N,1
1005592,33032,20220427112116,20220427,112305,2,MKID003,14665,NaN,Y,1
1005593,33032,20220427112116,20220427,112335,2,MKID003,14662,NaN,Y,1


In [53]:
hd.loc[hd['short_trailer'] == 0]

,profile_id,ss_id,date,time,time_slot,act_target_dtl,album_id,payment,continuous_play,short_trailer
0,3,20220301115653,20220301,115719,2,MKID003,15,NaN,Y,0
1,3,20220301115653,20220301,115809,2,MKID003,16,NaN,Y,0
2,3,20220301115653,20220301,115958,2,MKID003,17,NaN,Y,0
3,3,20220301115653,20220301,120118,3,MKID003,18,NaN,Y,0
4,3,20220301115653,20220301,120229,3,MKID003,19,NaN,Y,0
...,...,...,...,...,...,...,...,...,...,...
1005643,33032,20220427155091,20220427,155601,3,MKID003,381,NaN,Y,0
1005644,33032,20220427155091,20220427,155611,3,MKID003,381,NaN,Y,0
1005645,33032,20220427155091,20220427,155642,3,MKID003,381,NaN,N,0
1005648,33032,20220427155839,20220427,155810,3,MKID003,125,NaN,Y,0


#### 4-2. profile data

In [54]:
prd['sex'] = prd['sex'].apply(lambda x: 1 if x == 'M' else 0)

In [55]:
prd.head()

,profile_id,sex,age,pr_interest_keyword_cd_1,pr_interest_keyword_cd_2,pr_interest_keyword_cd_3,ch_interest_keyword_cd_1,ch_interest_keyword_cd_2,ch_interest_keyword_cd_3
0,3,0,5,P02,P04,P07,K01,K03,K04
1,5,1,5,P07,P08,P06,K05,K08,K09
2,7,0,9,P05,P03,P05,K06,K04,K06
3,12,1,6,P03,P06,P02,K09,K07,K03
4,16,0,12,P03,P06,P01,K01,K06,K04


In [56]:
for i in ['pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
          'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'] : 
    enc_1 = (prd.groupby(i).size()) / len(prd)
    enc_1
    
    prd['{}_encode'.format(i)] = prd[i].apply(lambda x : enc_1[x])
    
#인코딩 전 변수는 제거 
prd = prd.drop(['pr_interest_keyword_cd_1', 'pr_interest_keyword_cd_2', 'pr_interest_keyword_cd_3',
                'ch_interest_keyword_cd_1', 'ch_interest_keyword_cd_2', 'ch_interest_keyword_cd_3'],axis=1)

In [57]:
prd

,profile_id,sex,age,pr_interest_keyword_cd_1_encode,pr_interest_keyword_cd_2_encode,pr_interest_keyword_cd_3_encode,ch_interest_keyword_cd_1_encode,ch_interest_keyword_cd_2_encode,ch_interest_keyword_cd_3_encode
0,3,0,5,0.184093,0.093611,0.178438,0.403562,0.178799,0.105884
1,5,1,5,0.131512,0.078089,0.199374,0.051859,0.083023,0.089520
2,7,0,9,0.093731,0.149320,0.102394,0.062207,0.116111,0.104921
3,12,1,6,0.128865,0.211888,0.127903,0.033209,0.047527,0.157622
4,16,0,12,0.128865,0.211888,0.079293,0.403562,0.092167,0.105884
...,...,...,...,...,...,...,...,...,...
8306,33022,1,1,0.069787,0.093611,0.086271,0.088798,0.083023,0.105884
8307,33023,1,5,0.194080,0.149320,0.178438,0.063169,0.116111,0.095777
8308,33026,0,8,0.163037,0.149320,0.097341,0.051859,0.060643,0.104921
8309,33027,0,4,0.069787,0.106245,0.199374,0.135483,0.153652,0.095777


#### 5. meta data

In [58]:
import matplotlib.pyplot as plt
plt.rc('font', family = 'Malgun Gothic')
plt.rc('axes', unicode_minus = False)

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

#### 5-1. cast 1~7, genre small 삭제

In [59]:
md.head()

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
2,2131,타요의 첫 운행,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
3,2625,길 잃은 타요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
4,2594,새내기 꼬마 버스의 하루,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN


In [60]:
md.shape

(42602, 16)

In [61]:
# cast 1~7, genre small 삭제
md = md.drop(['genre_small', 'cast_1', 'cast_2',
       'cast_3', 'cast_4', 'cast_5', 'cast_6', 'cast_7'], axis = 1)

In [62]:
md.isnull().sum()

album_id           0
title              0
sub_title          0
genre_large        0
genre_mid          0
country         8868
run_time           0
onair_date     37258
dtype: int64

#### 5-2. onair데이터 : 7월 이후에 방영된 데이터 삭제

In [63]:
meta = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
md = meta.copy()

In [64]:
md = md.fillna({'onair_date': 0})  

In [65]:
md["onair_date"] = md["onair_date"].astype(str)
md["month"] = md["onair_date"].str.slice(0, 6)

In [66]:
print('7월 이후 방영된 건 수:', len(md.loc[md['month'] >= '202207']))

7월 이후 방영된 건 수: 21


In [67]:
# 이상 데이터 제거
md = md.drop(md.loc[md['month'] >= '202207'].index)

In [68]:
md = md.drop(['month'], axis = 1)

In [69]:
md.shape

(42581, 16)

#### 6. meta data plus

In [32]:
mpd.head()

,album_id,keyword_type,keyword_name,keyword_value
0,749,05010101,시리즈,0
1,749,05010302,EBS,0
2,749,05010401,3세,0
3,749,05010402,4세,0
4,749,05010407,1세,0


In [39]:
mpd.nunique()

album_id         39875
keyword_type      1101
keyword_name      1062
keyword_value        6
dtype: int64

In [71]:
# keyword_type과 keyword_name은 1대1 매칭
set(mpd.groupby('keyword_type')['keyword_name'].nunique())

{1}

In [78]:
mpd.groupby('keyword_type')['keyword_name'].nunique()

keyword_type
05010101    1
05010102    1
05010103    1
05010104    1
05010201    1
           ..
050c1901    1
050c1902    1
050c1903    1
050c1904    1
050c1905    1
Name: keyword_name, Length: 1101, dtype: int64

In [73]:
mpd.nunique()

album_id         39875
keyword_type      1101
keyword_name      1062
keyword_value        6
dtype: int64

In [75]:
mpd[mpd.duplicated([])]

,album_id,keyword_type,keyword_name,keyword_value


In [79]:
mpd.shape

(767948, 4)